# Example 32: Baseline Forecasting Models

**New in v1.0.0**: `null_model()` and `naive_reg()` for baseline forecasting

## Overview

This notebook demonstrates the NEW baseline forecasting models added in py-tidymodels v1.0.0:

### `null_model()` - Statistical Baselines
- **mean**: Average of historical values
- **median**: Median of historical values
- **last**: Last observed value (persistence)

### `naive_reg()` - Time Series Baselines
- **naive**: Last observation (same as null_model last)
- **seasonal_naive**: Value from same period last season
- **drift**: Linear trend from first to last observation
- **window**: Moving average over recent window

## When to Use Baseline Models

**Critical for model evaluation**:
1. Establish performance floor - any advanced model should beat baselines
2. Quick sanity checks - if complex model loses to mean baseline, investigate!
3. Production fallbacks - use baselines when advanced models fail
4. Benchmark comparisons - report "% improvement over naive forecast"

**Sometimes baselines are sufficient**:
- Stable time series with no trend/seasonality
- Limited historical data (< 50 observations)
- High forecast uncertainty makes complex models overkill
- Interpretability is paramount

## Dataset

**JODI Crude Oil Production** (Saudi Arabia):
- Monthly production from 2002-2024
- Units: Thousands of barrels per day (KBD)
- Major oil producer with stable production patterns
- Source: Joint Organisations Data Initiative

In [ ]:
# Setup
import pandas as pd
import numpy as np
from datetime import timedelta

# py-tidymodels imports
from py_parsnip import null_model, naive_reg, linear_reg, prophet_reg
from py_workflows import Workflow
from py_rsample import initial_time_split
from py_yardstick import rmse, mae, mape
from py_workflowsets import WorkflowSet
from py_yardstick import metric_set

import warnings
warnings.filterwarnings('ignore')

print("✓ Imports complete")

## 1. Load and Prepare Data

In [ ]:
# Load JODI crude production data
df = pd.read_csv('../_md/__data/jodi_crude_production_data.csv')
df['date'] = pd.to_datetime(df['date'])

# Filter to Saudi Arabia only
saudi = df[df['country'] == 'Saudi Arabia'].copy()
saudi = saudi[['date', 'value']].rename(columns={'value': 'production'})
saudi = saudi.sort_values('date').reset_index(drop=True)

# Remove zeros (missing months)
saudi = saudi[saudi['production'] > 0].reset_index(drop=True)

print(f"Saudi Arabia crude oil production:")
print(f"  Records: {len(saudi):,}")
print(f"  Date range: {saudi['date'].min()} to {saudi['date'].max()}")
print(f"  Mean production: {saudi['production'].mean():.0f} KBD")
print(f"  Std dev: {saudi['production'].std():.0f} KBD")
print(f"\nFirst few rows:")
print(saudi.head())

In [ ]:
# Train/test split (hold out last 12 months)
split = initial_time_split(saudi, date_column='date', prop=0.90)
train = split.training()
test = split.testing()

print(f"Train: {len(train)} months ({train['date'].min()} to {train['date'].max()})")
print(f"Test:  {len(test)} months ({test['date'].min()} to {test['date'].max()})")
print(f"\nTest period: {len(test)} months for evaluation")

## 2. null_model() - Statistical Baselines

These models use simple statistics from training data.

### 2.1 Mean Baseline

Forecast = mean of all training values

In [ ]:
# Mean baseline: predicts average of training data
mean_spec = null_model(strategy='mean')
mean_fit = mean_spec.fit(train, 'production ~ date')

# Predict on test
mean_eval = mean_fit.evaluate(test)
outputs_mean, coeffs_mean, stats_mean = mean_eval.extract_outputs()

# Extract test metrics (LONG format)
test_stats = stats_mean[stats_mean['split'] == 'test']
test_rmse = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_mape = test_stats[test_stats['metric'] == 'mape']['value'].iloc[0]

print("🎯 Null Model (Mean Strategy) - Test Performance:")
print(f"  Test RMSE: {test_rmse:.2f} KBD")
print(f"  Test MAE: {test_mae:.2f} KBD")
print(f"  Test MAPE: {test_mape:.2f}%")

### 2.2 Median Baseline

Forecast = median of all training values (robust to outliers)

In [ ]:
# Median baseline: more robust to outliers
median_spec = null_model(strategy='median')
median_fit = median_spec.fit(train, 'production ~ date')
median_eval = median_fit.evaluate(test)
_, _, stats_median = median_eval.extract_outputs()

# Extract test metrics (LONG format)
test_stats = stats_median[stats_median['split'] == 'test']
test_rmse_median = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_median = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_mape_median = test_stats[test_stats['metric'] == 'mape']['value'].iloc[0]

print("🎯 Null Model (Median Strategy) - Test Performance:")
print(f"  Test RMSE: {test_rmse_median:.2f} KBD")
print(f"  Test MAE: {test_mae_median:.2f} KBD")
print(f"  Test MAPE: {test_mape_median:.2f}%")

### 2.3 Last Value Baseline (Persistence)

Forecast = last observed training value

In [ ]:
# Last value: persistence forecast
last_spec = null_model(strategy='last')
last_fit = last_spec.fit(train, 'production ~ date')
last_eval = last_fit.evaluate(test)
_, _, stats_last = last_eval.extract_outputs()

# Extract test metrics (LONG format)
test_stats = stats_last[stats_last['split'] == 'test']
test_rmse_last = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_last = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_mape_last = test_stats[test_stats['metric'] == 'mape']['value'].iloc[0]

print("🎯 Null Model (Last Strategy) - Test Performance:")
print(f"  Test RMSE: {test_rmse_last:.2f} KBD")
print(f"  Test MAE: {test_mae_last:.2f} KBD")
print(f"  Test MAPE: {test_mape_last:.2f}%")

print(f"\n  (Should match 'last' from null_model)")

## 3. naive_reg() - Time Series Baselines

These models use time series specific patterns.

### 3.1 Naive Forecast

Same as persistence (last value)

In [ ]:
# Naive: last observation carried forward
naive_spec = naive_reg(strategy='naive')
naive_fit = naive_spec.fit(train, 'production ~ date')
naive_eval = naive_fit.evaluate(test)
_, _, stats_naive = naive_eval.extract_outputs()

# Extract test metrics (LONG format)
test_stats = stats_naive[stats_naive['split'] == 'test']
test_rmse_naive = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_naive = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_mape_naive = test_stats[test_stats['metric'] == 'mape']['value'].iloc[0]

print("🔄 Naive Regression (Naive Strategy) - Test Performance:")
print(f"  Test RMSE: {test_rmse_naive:.2f} KBD")
print(f"  Test MAE: {test_mae_naive:.2f} KBD")
print(f"  Test MAPE: {test_mape_naive:.2f}%")

### 3.2 Seasonal Naive

Forecast = value from same month last year (12-month seasonality)

In [ ]:
# Seasonal naive: value from same period last year
seasonal_spec = naive_reg(strategy='seasonal_naive', seasonal_period=12)
seasonal_fit = seasonal_spec.fit(train, 'production ~ date')
seasonal_eval = seasonal_fit.evaluate(test)
_, _, stats_seasonal = seasonal_eval.extract_outputs()

# Extract test metrics (LONG format)
test_stats = stats_seasonal[stats_seasonal['split'] == 'test']
test_rmse_seasonal = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_seasonal = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_mape_seasonal = test_stats[test_stats['metric'] == 'mape']['value'].iloc[0]

print("🔄 Naive Regression (Seasonal Naive) - Test Performance:")
print(f"  Test RMSE: {test_rmse_seasonal:.2f} KBD")
print(f"  Test MAE: {test_mae_seasonal:.2f} KBD")
print(f"  Test MAPE: {test_mape_seasonal:.2f}%")

### 3.3 Drift Method

Extrapolates linear trend from first to last training observation

In [ ]:
# Drift: linear extrapolation from first to last value
drift_spec = naive_reg(strategy='drift')
drift_fit = drift_spec.fit(train, 'production ~ date')
drift_eval = drift_fit.evaluate(test)
_, _, stats_drift = drift_eval.extract_outputs()

# Extract test metrics (LONG format)
test_stats = stats_drift[stats_drift['split'] == 'test']
test_rmse_drift = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_drift = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]

print("🔄 Naive Regression (Drift Strategy) - Test Performance:")
print(f"  Test RMSE: {test_rmse_drift:.2f} KBD")
print(f"  Test MAE: {test_mae_drift:.2f} KBD")

### 3.4 Window Average (Moving Average)

Forecast = average of last N observations

In [ ]:
# Window average: mean of last 6 months
window_spec = naive_reg(strategy='window', window_size=6)
window_fit = window_spec.fit(train, 'production ~ date')
window_eval = window_fit.evaluate(test)
_, _, stats_window = window_eval.extract_outputs()

# Extract test metrics (LONG format)
test_stats = stats_window[stats_window['split'] == 'test']
test_rmse_window = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_window = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]

print("🔄 Naive Regression (Window Strategy) - Test Performance:")
print(f"  Test RMSE: {test_rmse_window:.2f} KBD")
print(f"  Test MAE: {test_mae_window:.2f} KBD")

## 4. Baseline Comparison Framework

Compare all baselines side-by-side using WorkflowSet.

In [ ]:
# Create all baseline workflows
baseline_models = [
    ('mean', null_model(strategy='mean')),
    ('median', null_model(strategy='median')),
    ('last', null_model(strategy='last')),
    ('naive', naive_reg(strategy='naive')),
    ('seasonal_naive_12m', naive_reg(strategy='seasonal_naive', seasonal_period=12)),
    ('drift', naive_reg(strategy='drift')),
    ('window_6m', naive_reg(strategy='window', window_size=6)),
    ('window_3m', naive_reg(strategy='window', window_size=3)),
]

# Create workflows
baseline_workflows = []
for name, model in baseline_models:
    wf = Workflow().add_formula('production ~ date').add_model(model)
    baseline_workflows.append(wf)

# Create WorkflowSet
wf_set_baselines = WorkflowSet.from_workflows(baseline_workflows)

print(f"Created {len(baseline_workflows)} baseline workflows")
print(f"Workflow IDs: {list(wf_set_baselines.workflows.keys())}")

In [ ]:
# Compare all null_model strategies
baseline_comparison = pd.DataFrame([
    {'strategy': 'mean', 'rmse': test_rmse, 'mae': test_mae, 'mape': test_mape},
    {'strategy': 'median', 'rmse': test_rmse_median, 'mae': test_mae_median, 'mape': test_mape_median},
    {'strategy': 'last', 'rmse': test_rmse_last, 'mae': test_mae_last, 'mape': test_mape_last}
]).sort_values('rmse')

print("\n📊 Baseline Comparison (null_model strategies):")
print(baseline_comparison.to_string(index=False))

print(f"\n🏆 Best baseline strategy: {baseline_comparison.iloc[0]['strategy']}")
print(f"  RMSE: {baseline_comparison.iloc[0]['rmse']:.2f} KBD")
print(f"  MAE: {baseline_comparison.iloc[0]['mae']:.2f} KBD")

## 5. Baselines vs Advanced Models

Compare baselines against advanced models (Linear Regression and Prophet).

In [ ]:
# Add advanced models
advanced_models = [
    ('linear_reg', linear_reg()),
    ('prophet', prophet_reg())
]

# Create workflows for advanced models
all_workflows = baseline_workflows.copy()
for name, model in advanced_models:
    wf = Workflow().add_formula('production ~ date').add_model(model)
    all_workflows.append(wf)

# Create WorkflowSet with all models
wf_set_all = WorkflowSet.from_workflows(all_workflows)

print(f"Total workflows: {len(all_workflows)} (8 baselines + 2 advanced)")

In [ ]:
# Compare ALL strategies (null_model + naive_reg)
all_comparison = pd.DataFrame([
    {'type': 'baseline', 'strategy': 'mean', 'rmse': test_rmse, 'mae': test_mae, 'mape': test_mape},
    {'type': 'baseline', 'strategy': 'median', 'rmse': test_rmse_median, 'mae': test_mae_median, 'mape': test_mape_median},
    {'type': 'baseline', 'strategy': 'last', 'rmse': test_rmse_last, 'mae': test_mae_last, 'mape': test_mape_last},
    {'type': 'naive', 'strategy': 'naive', 'rmse': test_rmse_naive, 'mae': test_mae_naive, 'mape': test_mape_naive},
    {'type': 'naive', 'strategy': 'seasonal_naive', 'rmse': test_rmse_seasonal, 'mae': test_mae_seasonal, 'mape': test_mape_seasonal},
    {'type': 'naive', 'strategy': 'drift', 'rmse': test_rmse_drift, 'mae': test_mae_drift, 'mape': np.nan},
    {'type': 'naive', 'strategy': 'window', 'rmse': test_rmse_window, 'mae': test_mae_window, 'mape': np.nan}
]).sort_values('rmse')

print("\n📊 ALL Strategies Comparison:")
print(all_comparison.to_string(index=False))

# Calculate improvement
best_baseline_rmse = all_comparison[all_comparison['type']=='baseline']['rmse'].min()
all_comparison['improvement_vs_best_baseline'] = \
    (best_baseline_rmse - all_comparison['rmse']) / best_baseline_rmse * 100

print(f"\n🏆 Best overall strategy: {all_comparison.iloc[0]['strategy']}")
print(f"  Type: {all_comparison.iloc[0]['type']}")
print(f"  RMSE: {all_comparison.iloc[0]['rmse']:.2f} KBD")
print(f"  MAE: {all_comparison.iloc[0]['mae']:.2f} KBD")

In [ ]:
# Show improvement over best baseline
print("\n📈 Improvement vs Best Baseline:")
print(all_comparison[['strategy', 'type', 'rmse', 'improvement_vs_best_baseline']].to_string(index=False))

# Highlight improvements
print("\n🎯 Analysis:")
for _, row in all_comparison.iterrows():
    if row['improvement_vs_best_baseline'] > 0:
        print(f"  ✓ {row['strategy']}: {row['improvement_vs_best_baseline']:.1f}% better than best baseline")
    else:
        print(f"  ✗ {row['strategy']}: {abs(row['improvement_vs_best_baseline']):.1f}% WORSE than best baseline")

## 6. Key Takeaways

### When Each Baseline Works Best

1. **Mean/Median**: 
   - Stationary series with no trend
   - Random walk patterns
   - Long-term averages

2. **Naive/Last Value**:
   - Strong persistence (tomorrow ≈ today)
   - Short-term forecasts
   - Stable recent patterns

3. **Seasonal Naive**:
   - Strong seasonal patterns
   - Consistent year-over-year behavior
   - Retail sales, energy demand

4. **Drift**:
   - Clear linear trends
   - No seasonality
   - Steady growth/decline

5. **Window Average**:
   - Smoothing noisy data
   - Recent changes more important
   - Balancing stability vs responsiveness

### Best Practices

1. **Always establish baseline performance FIRST**
   - Run all baseline models before complex ones
   - Document baseline RMSE/MAE as reference

2. **Advanced models MUST beat baselines**
   - If linear_reg loses to mean, check for bugs
   - If Prophet loses to seasonal_naive, investigate why
   - Report "% improvement over naive forecast"

3. **Use baselines as production fallbacks**
   - If advanced model fails, fall back to seasonal_naive
   - Baselines never fail (no hyperparameters, no convergence issues)

4. **Choose baseline strategy based on data characteristics**
   - Check for trend → use drift
   - Check for seasonality → use seasonal_naive
   - Neither → use mean or window average

### Production Deployment

```python
# Production pattern: baseline fallback
try:
    predictions = advanced_model.predict(new_data)
except Exception as e:
    logger.warning(f"Advanced model failed: {e}")
    logger.info("Falling back to seasonal_naive baseline")
    predictions = baseline_model.predict(new_data)
```

### Common Pitfalls

1. **Seasonal period mismatch**
   - Weekly data: use seasonal_period=52 (not 12)
   - Daily data: use seasonal_period=7 or 365

2. **Window size too small/large**
   - Too small: noisy predictions
   - Too large: slow to react to changes
   - Rule of thumb: 3-12 observations

3. **Ignoring baseline performance**
   - If complex model only 2% better → use baseline (simpler)
   - If complex model 50% better → definitely worth complexity

## Summary

This notebook demonstrated:

✅ All `null_model()` strategies (mean, median, last)  
✅ All `naive_reg()` strategies (naive, seasonal_naive, drift, window)  
✅ Baseline comparison framework with WorkflowSet  
✅ Baselines vs advanced models (linear_reg, prophet_reg)  
✅ When to use each baseline strategy  
✅ Best practices for production deployment  

**Key Insight**: Baselines establish the performance floor. Any advanced model should beat the best baseline by a meaningful margin (10-20%+), otherwise the added complexity isn't justified.

**Next Steps**:
- Example 33: Recursive multistep forecasting
- Example 34: Gradient boosting engines comparison